In [45]:
# importing libraries
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import joblib

In [46]:
def preprocesses():
    """preprocesses the data:
    parameter:
    return :
    """
    df = pd.read_csv(r"D:\Arai4_Projects\kpmgProject\data\B\metadata_with_fr_nl_info.csv")
    #df = pd.read_csv('/data/B/metadata_with_fr_nl_info.csv')
    df = df[['text_nl','label']] # selecting feature and target for modeling
    df = df.astype(str)
    df = df.dropna()
    
    return df
    
    


In [47]:

def balance_sample_down(df):
    """downsample the data
     
    """
    
    # Separate majority and minority classes
    df_majority = df[df.label == "new"]
    df_minority = df[df.label == "modified"]
    
 
    #downsample majority class
    df_majority_downsampled = resample(df_majority, 
                                 replace=False,     # sample with replacement
                                 n_samples = 257,    # to match majority class
                                 random_state =123) # reproducible results
 
    # Combine majority class with upsampled minority class
    df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
    # Display new class counts
    print("Down sampled: ", df_downsampled.label.value_counts())

    y = df_downsampled.label
    X = df_downsampled['text_nl'].to_list()
    

    return X, y

In [48]:
def classifier_model():
    """classifier 
    """
    df = preprocesses()
    X,y = balance_sample_down(df)
    X_train, X_test, Y_train, Y_test = train_test_split(X,y,test_size=1/5,random_state=42) 
    
    # conveting to desired array fromat
    y_train =Y_train.to_numpy()
    y_train =np.reshape(y_train, (-1, 1))
    y_test = Y_test.to_numpy()
    y_test = np.reshape(y_test,(-1, 1))
    

    mlb_1 = MultiLabelBinarizer()
    y_train = mlb_1.fit_transform(y_train)
    y_test = mlb_1.fit_transform(y_test)
    # print(Y_1)
    # print(X_train)

    classifier = Pipeline([
         ('vectorizer', CountVectorizer()),
         ('tfidf', TfidfTransformer()),
         ('clf', MultiOutputClassifier(OneVsRestClassifier(LinearSVC())))])

    classifier.fit(X_train, y_train)
    predicted = classifier.predict(X_test)
    joblib.dump(classifier, 'label.pkl')
    #print("score", classifier.score(predicted,y_test))
    all_labels = mlb_1.inverse_transform(predicted)

#     print('text classifier:')
#     for item, labels in zip(X_test, all_labels):
#          print('{0} => {1}'.format(item, ', '.join(labels)))




classifier_model()
    

Down sampled:  modified    913
new         257
Name: label, dtype: int64


In [49]:
txt = """ --------------Page 1-----------------
AANVULLEND PARITAIR COMITE VOOR DE
WERKLIEDEN
COLLECTIEVE ARBEIDSOVEREENKOMST VAN 7
DECEMBER 2021
VRIJSTELLING VAN DE VERPLICHTING VAN
AANGEPASTE BESCHIKBAARHEID 2023 - 2024
HOOFDSTUK - TOEPASSINGSGEBIED
ART. 1
$ 1
Deze collectieve arbeidsovereenkomst is van toepas-
sing op de werkgevers en arbeiders van de onderne-
mingen die tot de bevoegdheid van het Aanvullend Pa-
ritair Comité voor de werklieden behoren.
§2.
Onderarbeiderswordt verstaan de mannelijke en
vrouwelijke arbeiders.
HOOFDSTUK II - DRAAGWIJDTE VAN DE OVER-
EENKOMST
ART. 2
81.
Deze collectieve arbeidsovereenkomst wordt gesloten
om een vrijstelling toe te passen van de verplichting
wen aangepaste beschikbaarheid ter uitvoering van
het sociaal akkoord in het kader van de interprofessi-
onele onderhandelingen voor de periode 2021-2022.
Deze regeling wordt georganiseerd voor de geldig-
heidsduur van deze collectieve arbeidsovereenkomst
door:
- de collectieve arbeidsovereenkomst nr. 155 van
de Nationale Arbeidsraad van 15 juli 2021 tot
vaststelling, voor 2023-2024, van de voorwaar-
den voor de toekenning van de vrijstelling van de
verplichting we.n aangepaste beschikbaarheid
voor oudere werknemers die worden ontslagen
voor 1 juli 2023 in het raam van een stelsel van
werkloosheid met bedrijfstoeslag, die 20 jaar heb-
ben gewerkt in een regeling van nachtarbeid, die
hebben gewerkt in een zwaar beroep of tewerk-
gesteld werden in het bouwbedrijf en arbeidson-
geschikt zijn, die hebben gewerkt in een zwaar
beroep en 35 jaar beroepsverleden aantonen of
een lange loopbaan hebben, of die tewerkgesteld
waren in een onderneming in moeilijkheden of in
herstructurering (nr. 166.894/CO)

"""

In [54]:
predicted_label =joblib.load('label.pkl').predict([txt])
predicted_label



array([[0, 1]])